In [24]:
import hvplot.pandas
import hvplot
import pandas as pd
import requests
import geoviews as gv
import geoviews.feature as gf
import xarray as xr
from cartopy import crs
import warnings
warnings.filterwarnings("ignore")

In [2]:
city_data = pd.read_csv("Cities_database/Cities_database.csv")
city_data.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Esperance,AU,-33.8667,121.9000,72.95,72,0,18.77,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,47.86,87,100,23.02,light rain
2,2,Provideniya,RU,64.3833,-173.3000,-3.89,59,40,11.18,scattered clouds
3,3,Kodiak,US,57.7900,-152.4072,33.31,69,40,3.44,scattered clouds
4,4,Norman Wells,CA,65.2820,-126.8329,-11.16,70,100,4.61,light snow


In [3]:
#Map of every City
map_plot = city_data.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
#Narrow Search
city_data_2 = city_data[city_data['Max Temp']>63]

In [5]:
city_data_2_1 = city_data_2[city_data_2['Max Temp']<78]

In [6]:
city_data_2_2 = city_data_2_1[city_data_2_1['Wind Speed']<5]

In [7]:
city_data_2_3 = city_data_2_2[city_data_2_2['Current Description']=='clear sky']

In [8]:
city_data_2_3.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
33,33,Taltal,CL,-25.4000,-70.4833,70.83,80,9,2.62,clear sky
37,37,New Norfolk,AU,-42.7826,147.0587,68.90,35,1,4.00,clear sky
91,91,Chuy,UY,-33.6971,-53.4616,68.13,81,2,2.64,clear sky
183,183,Les Cayes,HT,18.2000,-73.7500,73.44,82,2,4.79,clear sky
188,188,Vallenar,CL,-28.5708,-70.7581,69.85,67,3,4.12,clear sky


In [9]:
#Check for any null values
city_data_2_3.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
#HOTELS
hotels_df = city_data_2_3[['City','Country','Lat','Lng','Humidity']].copy()

In [11]:
hotels_df['Hotel']=" "
hotels_df.head()

,City,Country,Lat,Lng,Humidity,Hotel
33,Taltal,CL,-25.4000,-70.4833,80,
37,New Norfolk,AU,-42.7826,147.0587,35,
91,Chuy,UY,-33.6971,-53.4616,81,
183,Les Cayes,HT,18.2000,-73.7500,82,
188,Vallenar,CL,-28.5708,-70.7581,67,


In [16]:
#Finding Hotels
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey":"53cbd3c1f6d242f59e5c93f081b110d5"}

for index, row in hotels_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    base_url = "https://api.geoapify.com/v2/places?PARAMS"
    
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotels_df.loc[index, "Hotel"] = hotels["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotels_df.loc[index, "Hotel"] = "No hotel found"
    
    print(f"{hotels_df.loc[index, 'City']} - nearest hotel: {hotels_df.loc[index, 'Hotel']}")
    
hotels_df

Taltal - nearest hotel: Hostería Taltal
New Norfolk - nearest hotel: Woodbridge on the Derwent
Chuy - nearest hotel: Alerces
Les Cayes - nearest hotel: No hotel found
Vallenar - nearest hotel: Hotel del Marqués
Machali - nearest hotel: Hotel Boutique Il Giardino
East London - nearest hotel: No hotel found
Rocha - nearest hotel: Hotel Municipal
Vitoria Da Conquista - nearest hotel: Faixa Hotel
Vila Velha - nearest hotel: Hotel Prainha
Moro - nearest hotel: No hotel found
Pinotepa Nacional - nearest hotel: No hotel found
Thenzawl - nearest hotel: No hotel found
Rafai - nearest hotel: No hotel found
Salta - nearest hotel: Ayres de Salta Hotel
Takaka - nearest hotel: No hotel found
Correntina - nearest hotel: Pousada Silva
La Palma - nearest hotel: La Quinta Inn & Suites by Wyndham Buena Park
Ouadda - nearest hotel: No hotel found
Chor - nearest hotel: No hotel found
Paithan - nearest hotel: No hotel found
Richards Bay - nearest hotel: Protea Hotel


,City,Country,Lat,Lng,Humidity,Hotel
33,Taltal,CL,-25.4000,-70.4833,80,Hostería Taltal
37,New Norfolk,AU,-42.7826,147.0587,35,Woodbridge on the Derwent
91,Chuy,UY,-33.6971,-53.4616,81,Alerces
183,Les Cayes,HT,18.2000,-73.7500,82,No hotel found
188,Vallenar,CL,-28.5708,-70.7581,67,Hotel del Marqués
192,Machali,CL,-34.1833,-70.6667,44,Hotel Boutique Il Giardino
197,East London,ZA,-33.0153,27.9116,94,No hotel found
264,Rocha,UY,-34.4833,-54.3333,97,Hotel Municipal
283,Vitoria Da Conquista,BR,-14.8661,-40.8394,82,Faixa Hotel
313,Vila Velha,BR,-20.3297,-40.2925,90,Hotel Prainha


In [20]:
vacation_hotels_df = hotels_df[hotels_df['Hotel'] != "No hotel found"].copy()
vacation_hotels_df

,City,Country,Lat,Lng,Humidity,Hotel
33,Taltal,CL,-25.4000,-70.4833,80,Hostería Taltal
37,New Norfolk,AU,-42.7826,147.0587,35,Woodbridge on the Derwent
91,Chuy,UY,-33.6971,-53.4616,81,Alerces
188,Vallenar,CL,-28.5708,-70.7581,67,Hotel del Marqués
192,Machali,CL,-34.1833,-70.6667,44,Hotel Boutique Il Giardino
264,Rocha,UY,-34.4833,-54.3333,97,Hotel Municipal
283,Vitoria Da Conquista,BR,-14.8661,-40.8394,82,Faixa Hotel
313,Vila Velha,BR,-20.3297,-40.2925,90,Hotel Prainha
395,Salta,AR,-24.7859,-65.4117,84,Ayres de Salta Hotel
524,Correntina,BR,-13.3433,-44.6367,77,Pousada Silva


In [22]:
#Saving the vacation hotel data
output_data_file_2 = "Cities_database/Vacation_Hotels.csv"
vacation_hotels_df.to_csv(output_data_file_2, index_label="City")

In [26]:
#Output map of places and hotels
map_plot_2 = vacation_hotels_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "Hotel",
    hover_cols=["Hotel", "Lat", "Lng", "City","Country", "Humidity" ]
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel,Humidity,City,Country)